In [2]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

Step 1: Load & Preprocess the Image

In [ ]:
def load_and_preprocess(image_path: str):
    # Read the image from the specified path using OpenCV.
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found. Please check the path.")
        
    # Convert the image from BGR (default in OpenCV) to grayscale.
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to smooth the image and reduce noise.
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    
    # Use adaptive thresholding to convert the blurred grayscale image into a binary image.
    # THRESH_BINARY_INV inverts the result so the signature (usually dark) becomes white.
    binary_image = cv2.adaptiveThreshold(blurred_image, 255,
                                         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY_INV, 11, 2)
    
    # Remove small noise from the binary image using morphological opening.
    # A kernel of size 3x3 is used for this operation.
    cleaned_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    # Return the original image (useful for later extraction) and the cleaned binary image.
    return image, cleaned_image

Step 2: Extract Signature

In [5]:
def extract_signature(original_image, binary_image):
    # Find all external contours in the binary image.
    # Contours represent continuous boundaries around connected components.
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Choose the largest contour based on area, assuming it represents the signature.
    signature_contour = max(contours, key=cv2.contourArea)
    
    # Create a mask of the same dimensions as the binary image, filled with zeros (black).
    mask = np.zeros_like(binary_image)
    # Draw the detected signature contour onto the mask.
    # Filling the contour ensures that the entire area of the signature is covered.
    cv2.drawContours(mask, [signature_contour], -1, 255, thickness=cv2.FILLED)
    
    # Use the mask to extract the signature from the original image.
    # This performs a bitwise 'AND' operation, keeping only the pixels where the mask is white.
    signature_image = cv2.bitwise_and(original_image, original_image, mask=mask)
    
    # Convert the extracted signature image from BGR to RGB for correct color display in matplotlib.
    signature_image = cv2.cvtColor(signature_image, cv2.COLOR_BGR2RGB)
    
    return signature_image

Step 3: Display and Save Results

In [6]:
def display_and_save(signature_image, output_path):
    # Display the signature image using matplotlib.
    plt.figure(figsize=(5, 5))
    plt.imshow(signature_image)
    plt.axis('off')  # Turn off axis ticks and labels for a cleaner look.
    plt.show()
    
    # Save the signature image to the specified output path using PIL.
    Image.fromarray(signature_image).save(output_path)
    print(f"Signature saved successfully at {output_path}")